In [1]:
import math
import time
import numpy as np
import pycuda.autoinit
import pycuda.driver as drv
from pycuda import compiler

OPT_TYPE_CALL = 0
OPT_TYPE_PUT = 1

In [2]:
opt_type = OPT_TYPE_CALL
spot_prc = 274.34
strike_prc = 275.0
remain_day = 31
interest_rate = 1.55
volatility = 13.59
node_cnt = 49

In [3]:
st_time = time.time()

remain_year = remain_day / 365
delta_t = max(remain_year / node_cnt, 0.000001)
a = math.exp(interest_rate * 0.01 * delta_t)
u = math.exp(volatility * 0.01 * math.sqrt(delta_t))
d = 1 / u
p = (a - d) / (u - d)
r_p = 1 - p

datas = [0] * (node_cnt + 1)

for row in range(node_cnt+1):
    datas[row] = spot_prc * (u**(node_cnt-row)) * (d**row)
    datas[row] = max(datas[row]-strike_prc, 0) if OPT_TYPE_CALL == opt_type else max(strike_prc-datas[row], 0)

for col in range(node_cnt, 0, -1):
    for row in range(col):
        datas[row] = (p * datas[row] + r_p * datas[row+1]) / a

ed_time = time.time()

print("소요시간(초)", ed_time - st_time)
print("옵션가", datas[0])

소요시간(초) 0.0009953975677490234
옵션가 4.203350070605945


In [4]:
# Kernel code
kernel_code = """
__constant__ int opt_type;
__constant__ float params[7];

__global__ void get_opt_prc_n_thd(float* ret) {
	extern __shared__ float datas[];

	int node_cnt = blockDim.x - 1;
	int tid = threadIdx.x;

	const float spot_prc = params[0];
	const float strike_prc = params[1];
	const float a = params[2];
	const float u = params[3];
	const float d = params[4];
	const float p = params[5];
	const float r_p = params[6];

	const float final_spot_prc = spot_prc * powf(u, node_cnt-tid) * powf(d, tid);
	const float payoff = (0==opt_type) ? fmaxf(final_spot_prc-strike_prc, 0) : fmaxf(strike_prc-final_spot_prc, 0);
	datas[tid] = payoff;
	
	bool has_offset = false;
	int offset = 0, pre_offset = 0;

	__syncthreads();

	for ( int col=node_cnt-1 ; col>=0 ; col-- ) {
		if ( tid <= col ) {
			has_offset = !has_offset;
		
			if ( has_offset ) {
				offset = blockDim.x;
				pre_offset = 0;
			}
			else {
				offset = 0;
				pre_offset = blockDim.x;
			}
		
			datas[tid + offset] = (p * datas[tid + pre_offset] + r_p * datas[tid + 1 + pre_offset]) / a;
		}

		__syncthreads();
	}

	if ( 0 == tid ) {
		offset = has_offset ? blockDim.x : 0;
		ret[0] = datas[offset];
	}
}
"""

# Compile the kernel code
mod = compiler.SourceModule(kernel_code)

st_time = time.time()

remain_year = remain_day / 365
delta_t = max(remain_year / node_cnt, 0.000001)
a = math.exp(interest_rate * 0.01 * delta_t)
u = math.exp(volatility * 0.01 * math.sqrt(delta_t))
d = 1 / u
p = (a - d) / (u - d)
r_p = 1 - p

params = [spot_prc, strike_prc, a, u, d, p, r_p]

inputs_int = np.array([opt_type], dtype=np.int)
inputs_float = np.array(params, dtype=np.float32)

drv.memcpy_htod(mod.get_global("opt_type")[0], inputs_int)
drv.memcpy_htod(mod.get_global("params")[0], inputs_float)

# Get kernel function
get_opt_func = mod.get_function("get_opt_prc_n_thd")

# Run & Print
thd_cnt = node_cnt + 1
result = np.zeros(1, dtype=np.float32)
shard_mem_size = 2 * thd_cnt * result.dtype.itemsize
get_opt_func(drv.InOut(result), block=(thd_cnt, 1, 1), shared=shard_mem_size)

ed_time = time.time()

print("소요시간(초)", ed_time - st_time)
print("옵션가", result[0])

소요시간(초) 0.0019991397857666016
옵션가 4.2031155


In [5]:
def get_d1(s, x, t, r, v):
	return (math.log(s / x) + (r + v * v * 0.5) * t) / (v * math.sqrt(t))


def get_d2(s, x, t, r, v):
	return (math.log(s / x) + (r - v * v * 0.5) * t) / (v * math.sqrt(t))


def get_nm_cdf(x):
	a1 = 0.31938153
	a2 = -0.356563782
	a3 = 1.781477937
	a4 = -1.821255978
	a5 = 1.330274429

	abs_x = math.fabs(x)
	k = 1.0 / (1.0 + 0.2316419 * abs_x)
	n_x = 1.0 - (math.exp(-1*abs_x*abs_x*0.5) / math.sqrt(2*math.pi)) * (a1*k + a2*k*k + a3*k*k*k + a4*k*k*k*k + a5*k*k*k*k*k)

	ret = 0

	if 0 > x:
		ret = 1.0 - n_x
	else:
		ret = n_x

	return ret


def get_bs_opt_prc(o_type, s, x, t, r, v):
	d1 = get_d1(s, x, t, r, v)
	d2 = get_d2(s, x, t, r, v)

	if OPT_TYPE_CALL == o_type:
		opt_prc = s * get_nm_cdf(d1) - x * math.exp(-1 * r * t) * get_nm_cdf(d2)
	else:
		opt_prc = x * math.exp(-1 * r * t) * get_nm_cdf(-1 * d2) - s * get_nm_cdf(-1 * d1)

	return opt_prc


bs_opt_prc = get_bs_opt_prc(
    opt_type,
    spot_prc,
    strike_prc,
    remain_day / 365,
    interest_rate * 0.01,
    volatility * 0.01
)

print("옵션가", bs_opt_prc)

옵션가 4.189248806121185
